In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Just_normal/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'YZV_data.csv'
YZV_data = pd.read_csv(path + filename)

In [ ]:
set_index(YZV_data)

In [ ]:
data = pd.DataFrame(columns=['garch', 'return'], index=YZV_data.index)

In [ ]:
data['garch'] = YZV_data["('GARCH', 'normal', 0)"].values

In [ ]:
data['return'] = YZV_data["log_returns"].values

In [ ]:
data['YZV'] = YZV_data["YZV"].values

In [ ]:
data

,garch,return,YZV
Date,,,
2000-02-02,0.800313,-0.027564,1.050442
2000-02-03,0.655577,0.893688,1.085892
2000-02-04,0.491617,-0.052641,0.965063
2000-02-07,0.408321,-0.025746,0.949015
2000-02-08,0.339040,0.976294,0.923597
...,...,...,...
2024-08-26,0.093281,-0.275848,0.452946
2024-08-27,-0.162765,0.111656,0.443623
2024-08-28,-0.126100,-0.507014,0.416314


# Split train, validation and test data

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [ ]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## YZV

In [ ]:
daily_train, daily_val, daily_test = split_data(data, 'YZV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
YZV_X_train, YZV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
YZV_X_val, YZV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
YZV_X_test, YZV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
# YZV_X_train = YZV_X_train.reshape(YZV_X_train.shape[0], win_sz, -1)
# YZV_X_val = YZV_X_val.reshape(YZV_X_val.shape[0], win_sz, -1)
# YZV_X_test = YZV_X_test.reshape(YZV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(YZV_X_train.shape[1], YZV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,037 (54.83 KB)

 Trainable params: 14,037 (54.83 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(YZV_X_train, YZV_y_train, epochs=80, batch_size=32, validation_data=(YZV_X_val, YZV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - loss: 0.2230 - val_loss: 0.1228
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.1378 - val_loss: 0.1792
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.1288 - val_loss: 0.1563
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.1149 - val_loss: 0.1861
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.1065 - val_loss: 0.1921
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.0977 - val_loss: 0.1937
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 0.0941 - val_loss: 0.1592
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0838 - val_loss: 0.1451
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0760 - val_loss: 0.1702
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0818 - val_loss: 0.2048
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0648 - val_loss: 0.2174
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Predictions
YZV_test_predictions = lstm_garch_model.predict(YZV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [ ]:
mse = mean_squared_error(YZV_y_test, YZV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(YZV_y_test, YZV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(YZV_y_test, YZV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 0.10748232003370402
RMSE: 0.3278449634106097
MAE: 0.2470701667697066
MAPE: 171.99338067583136%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(YZV_X_train.shape[1], YZV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │           7,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,741 (41.96 KB)

 Trainable params: 10,741 (41.96 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru_history = gru_model.fit(YZV_X_train, YZV_y_train, epochs=80, batch_size=32, validation_data=(YZV_X_val, YZV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - loss: 0.2550 - val_loss: 0.1381
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - loss: 0.1338 - val_loss: 0.1594
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 0.1361 - val_loss: 0.1724
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - loss: 0.0998 - val_loss: 0.1457
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 0.0966 - val_loss: 0.1849
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 0.0759 - val_loss: 0.1962
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.0658 - val_loss: 0.1742
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0586 - val_loss: 0.1772
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.0553 - val_loss: 0.1826
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.0459 - val_loss: 0.1656
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0537 - val_loss: 0.1575
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [34]:
# Predictions
YZV_gru_predictions = gru_model.predict(YZV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [35]:
mse = mean_squared_error(YZV_y_test, YZV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(YZV_y_test, YZV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(YZV_y_test, YZV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.13403024275317615
RMSE: 0.3661014104769007
MAE: 0.29807182782012787
MAPE: 225.67784194770817%


### LSTM_GRU

In [36]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(YZV_X_train.shape[1], YZV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
lstm_gru_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,045 (50.96 KB)

 Trainable params: 13,045 (50.96 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
lstm_gru_history = lstm_garch_model.fit(YZV_X_train, YZV_y_train, epochs=80, batch_size=32, validation_data=(YZV_X_val, YZV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0188 - val_loss: 0.1743
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0211 - val_loss: 0.2102
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0194 - val_loss: 0.1865
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0210 - val_loss: 0.1998
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - loss: 0.0154 - val_loss: 0.1990
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0157 - val_loss: 0.1777
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0190 - val_loss: 0.1756
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0164 - val_loss: 0.1816
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0164 - val_loss: 0.1927
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0166 - val_loss: 0.1895
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0180 - val_loss: 0.1946
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [39]:
# Predictions
YZV_lstm_gru_predictions = lstm_garch_model.predict(YZV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [40]:
mse = mean_squared_error(YZV_y_test, YZV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(YZV_y_test, YZV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(YZV_y_test, YZV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.11103187672542596
RMSE: 0.3332144605587008
MAE: 0.2485741766857402
MAPE: 127.71499577040959%
